<a href="https://colab.research.google.com/github/FeloXbit/Electric-School-Bus-Adoption-Analytics/blob/main/01_data_ingestion/fetch_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install requests pandas

In [ ]:
CKAN_URL = "https://datasets.wri.org/api/3/action/package_show?id=electric-school-bus-adoption"
API_ENDPOINT = f"{CKAN_URL}/api/3/action/"
!curl -I {API_ENDPOINT}package_list

HTTP/2 404 
date: Wed, 26 Mar 2025 13:57:43 GMT
content-type: application/json;charset=utf-8
content-length: 173
cache-control: public, max-age=0, must-revalidate
set-cookie: ckan=c6ff9f4957ef25fd7b96b5d8ce7bbae3f23e8f0ac07438ff4bc145f3b719a2af1aa98384; expires=Tue, 19-Jan-2038 03:14:07 GMT; HttpOnly; Path=/; SameSite=Lax
strict-transport-security: max-age=31536000; includeSubDomains
x-content-type-options: nosniff
x-xss-protection: 1
referrer-policy: origin
expect-ct: max-age=86400, enforce, report-uri='[reportURL]'
x-permitted-cross-domain-policies: none
cf-cache-status: DYNAMIC
report-to: {"endpoints":[{"url":"https:\/\/a.nel.cloudflare.com\/report\/v4?s=JKilPOP6lHcGevUE2g6SpI1yOI3rS4Cs5JFhZu5r8zyNN3H1sfzBKnCpVbncADH3T%2BBir%2Bu1XG7OUDEXfhsRfc7ffWXdTWvrIVCzxq%2BDRkam6wcBwhfBJgd8UmVREdOMcg9T"}],"group":"cf-nel","max_age":604800}
nel: {"success_fraction":0,"report_to":"cf-nel","max_age":604800}
server: cloudflare
cf-ray: 9267289eac18ba0c-SEA
alt-svc: h3=":443"; ma=86400
server-timing:

In [1]:
import requests
import os
from concurrent.futures import ThreadPoolExecutor
from tqdm import tqdm  # For progress bars (install with: pip install tqdm)

def fetch_wri_dataset(dataset_id):
    """Fetches dataset metadata from WRI's CKAN API"""
    API_URL = "https://datasets.wri.org/api/3/action/"
    endpoint = f"{API_URL}package_show?id={dataset_id}"

    try:
        response = requests.get(endpoint, timeout=10)
        response.raise_for_status()
        data = response.json()

        if not data.get("success", False):
            error_msg = data.get('error', {}).get('message', 'Unknown error')
            print(f"CKAN API error: {error_msg}")
            return None

        return data["result"]

    except requests.exceptions.RequestException as e:
        print(f"Request failed: {e}")
        return None
    except (ValueError, KeyError) as e:
        print(f"Data parsing error: {e}")
        return None

def download_resource(resource, dataset_name):
    """Downloads a single resource with error handling"""
    try:
        filename = f"{dataset_name}_{resource['id']}.{resource['format'].lower()}"

        # Skip if file already exists
        if os.path.exists(filename):
            print(f"Skipping existing file: {filename}")
            return filename

        print(f"Downloading: {resource['name']} ({resource['format']})")
        response = requests.get(resource['url'], stream=True, timeout=30)
        response.raise_for_status()

        # Handle large files with streaming
        with open(filename, 'wb') as f:
            for chunk in response.iter_content(chunk_size=8192):
                f.write(chunk)

        return filename
    except Exception as e:
        print(f"Failed to download {resource.get('name', 'unnamed')}: {str(e)[:100]}...")
        return None

def download_all_resources(dataset):
    """Downloads all resources in parallel"""
    if not dataset or 'resources' not in dataset:
        print("No resources found in dataset")
        return []

    # Create download directory
    os.makedirs("ckan_downloads", exist_ok=True)
    os.chdir("ckan_downloads")

    # Download with thread pool (max 5 concurrent downloads)
    with ThreadPoolExecutor(max_workers=5) as executor:
        futures = []
        for resource in dataset['resources']:
            futures.append(
                executor.submit(
                    download_resource,
                    resource,
                    dataset['name']
                )
            )

        # Track progress
        downloaded_files = []
        for future in tqdm(futures, desc="Downloading resources"):
            result = future.result()
            if result:
                downloaded_files.append(result)

    return downloaded_files

# Main execution
if __name__ == "__main__":
    dataset = fetch_wri_dataset("electric-school-bus-adoption")

    if dataset:
        print(f"\nDataset: {dataset.get('title', 'Untitled')}")
        print(f"Resources: {len(dataset.get('resources', []))}")

        downloaded_files = download_all_resources(dataset)

        print("\nDownload summary:")
        print(f"Successfully downloaded: {len(downloaded_files)}/{len(dataset.get('resources', []))}")
        print("Files saved in: ckan_downloads/")
    else:
        print("Failed to fetch dataset metadata")



Dataset: Dataset of Electric School Bus Adoption in the United States
Resources: 8
Downloading: Electric School Bus Adoption Version 1 (ZIP)
Downloading: Electric School Bus Adoption Version 2 (ZIP)
Downloading: Electric School Bus Adoption Version 3 (ZIP)
Downloading: Electric School Bus Adoption Version 4 (ZIP)
Downloading: Electric School Bus Adoption Version 5 (ZIP)


Downloading: Electric School Bus Adoption Version 6 (ZIP)
Downloading: Electric School Bus Adoption Version 7 (ZIP)
Downloading: Electric School Bus Adoption Version 8 (ZIP)



Download summary:
Successfully downloaded: 8/8
Files saved in: ckan_downloads/
